In [ ]:
# Import libraries
import boto3
import pandas as pd
import time
import plotly.express as px
import os

In [ ]:
# Configurations
DATABASE = "default"
TABLE = "btc_etl_table"
S3_OUTPUT = os.environ.get("ATHENA_S3_OUTPUT", "s3://861115334572-athena/Unsaved/2025")

In [ ]:
# Athena client
athena = boto3.client("athena")

In [ ]:
# Query to fetch the data
query = f"SELECT timestamp, CAST(btc_usd AS DOUBLE) as btc_usd FROM {DATABASE}.{TABLE} ORDER BY timestamp"

In [ ]:
# Start the query
response = athena.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": DATABASE},
    ResultConfiguration={"OutputLocation": S3_OUTPUT},
)
query_execution_id = response["QueryExecutionId"]

# Wait for the query to finish
while True:
    result = athena.get_query_execution(QueryExecutionId=query_execution_id)
    state = result["QueryExecution"]["Status"]["State"]
    if state in ["SUCCEEDED", "FAILED", "CANCELLED"]:
        break
    time.sleep(2)

if state != "SUCCEEDED":
    raise Exception(f"Query Athena falhou: {state}")

In [ ]:
# Download the result into a DataFrame
# Get the CSV path in S3
s3_csv_path = f"{S3_OUTPUT}{query_execution_id}.csv"

# Download the CSV from S3
s3 = boto3.client("s3")
bucket = S3_OUTPUT.replace("s3://", "").split("/")[0]
key = "/".join(S3_OUTPUT.replace("s3://", "").split("/")[1:] + [f"{query_execution_id}.csv"])
local_csv = "athena_result.csv"
s3.download_file(bucket, key, local_csv)

In [ ]:
# Read the CSV
df = pd.read_csv(local_csv)
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Plot the interactive chart
fig = px.line(
    df,
    x="timestamp",
    y="btc_usd",
    title="BTC Price Evolution (USD)",
    labels={"timestamp": "Datetime", "btc_usd": "BTC Price"}
)
fig.show() 